In [285]:
import tensorflow as tf
from tensorflow import keras
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense

from PIL import Image
from pylab import *
import cv2
from scipy.ndimage import  filters

In [286]:
train_dir = '/Users/nick/Downloads/chess_data/train'
val_dir = '/Users/nick/Downloads/chess_data/val'
test_dir = '/Users/nick/Downloads/chess_data/test'
#data_dir = '/Users/nick/Downloads/chess_data/data'

#классы
black_pawn = 'black_pawn'
empty = 'empty'
img_width, img_height = 720, 720
input_shape = (img_width, img_height, 3)
epochs = 10
batch_size = 14

# test_data_portion = 0.15
# val_data_portion = 0.15
# nb_images = 60

nb_train_samples = 137
nb_validation_samples = 19
nb_test_samples = 31

sigma = 0.45

In [287]:
# test_image = cv2.imread('/Users/nick/Downloads/chess_data/current_test/test_image.jpg', 3)
# cv2.resize(test_image, (720, 720))

In [288]:
# black_pawns = 'black_pawns'
# empty = 'empty'
# start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
# start_test_data_idx = int(nb_images * (1 - test_data_portion))
# print(start_test_data_idx, start_val_data_idx)# def copy_images(start_index, end_index, source_dir, dest_dir, chess_class):
#     i = 0
#     for fn in os.listdir(source_dir):
#         if fn == '.DS_Store':
#             continue
#         if i < start_index:
#             continue
#         if i >= end_index:
#             break
#         shutil.copy2(source_dir, os.path.join(dest_dir, chess_class))
#         i += 1
# copy_images(0, start_val_data_idx, data_dir, val_data, black_pawns)

In [289]:
def process_images(folder_name, class_name):
    for fn in os.listdir(folder_name + '/' + class_name):
        if fn == '.DS_Store':
            continue
        im = np.array(Image.open(os.path.join(folder_name + '/' + class_name + '/' + fn)).convert('L'))
        im = cv2.resize(im, (720, 720))
#         imx = np.zeros(im.shape)
#         filters.gaussian_filter(im, (sigma, sigma), (0, 1), imx)

#         imy = np.zeros(im.shape)
#         filters.gaussian_filter(im, (sigma, sigma), (1, 0), imy)

#         magn = np.sqrt(imx**2 + imy**2)
#         magn = np.uint8(magn)

        cv2.imwrite(os.path.join(os.path.join(folder_name + '/' + class_name + '/' + fn)), im)

#         plt.figure(figsize=(10,10))
#         plt.xticks([])
#         plt.yticks([])
#         plt.imshow(magn, cmap=plt.cm.binary)
#         plt.show()

In [290]:
process_images(train_dir, black_pawn)
process_images(train_dir, empty)

process_images(test_dir, black_pawn)
process_images(test_dir, empty)

process_images(val_dir, black_pawn)
process_images(val_dir, empty)

In [291]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [292]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
# current_test =. datagen.flow_from_directory(
#     current_test_dir,
#     targer_size)

Found 137 images belonging to 2 classes.
Found 19 images belonging to 2 classes.
Found 31 images belonging to 2 classes.


In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10


In [271]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Вероятность распознавания на тестовых данных: %.2f%%" % (scores[1]*100))

Вероятность распознавания на тестовых данных: 58.06%


In [242]:
#ахует

In [284]:
pred = model.predict(test_image)

ValueError: Error when checking input: expected conv2d_42_input to have 4 dimensions, but got array with shape (1600, 1586, 3)